In [2]:
import os
import psycopg2
from psycopg2.extras import RealDictCursor
from dotenv import load_dotenv
import hashlib
import hmac
import jwt
import os
import boto3
from botocore.exceptions import ClientError
import json
from dotenv import load_dotenv
from datetime import datetime, date
from decimal import Decimal
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
load_dotenv()

_conn = None
def get_secret(secret_name, region_name="eu-central-1"):
    """
    Retrieve a secret from AWS Secrets Manager
    """
    # Create a Secrets Manager client
    client = boto3.client("secretsmanager", region_name=region_name)

    try:
        response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e
    else:
        # Secret is stored either as string or binary
        if 'SecretString' in response:
            try:
                return json.loads(response['SecretString'])
            except Exception:
                return response['SecretString']
        else:
            import base64
            return json.loads(base64.b64decode(response['SecretBinary']))

def get_connection():
    global _conn
    db_secret_name = os.getenv("DB_SECRET_NAME","rds!db-efc52989-89c8-4009-a2c3-e211a33ba1bd")
    
    db_secret = get_secret(db_secret_name)
    if _conn is None or _conn.closed:
        _conn = psycopg2.connect(
            host= "cardy-dev.cb60yy2s4a4i.eu-central-1.rds.amazonaws.com",
            database= "budget",
            user="postgres",
            password=db_secret["password"],
            cursor_factory=RealDictCursor
        )
    return _conn

def execute_query(query, params=None, fetch=True, commit=False):
    conn = get_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(query, params)
            result = cur.fetchall() if fetch else None
        if commit:
            conn.commit()
        return result
    except Exception as e:
        # Rollback to reset transaction state
        conn.rollback()
        raise

In [3]:
def get_entries(user_id,scenario_id):
    # Fetch entries
    query = """
        SELECT
            e.id            AS entry_id,
            e.name          AS entry_name,
            e.type          AS entry_type,
            e.frequency     AS entry_frequency,
            e.start_date    AS entry_start_date,
            e.end_date      AS entry_end_date,
            e.amount        AS entry_amount,
            e.scenario_id   AS entry_scenario_id,
            e.category_id   AS entry_category_id,
            c.id            AS category_id,
            c.name          AS category_name
        FROM entries AS e
        JOIN category AS c
            ON e.category_id = c.id
        WHERE e.user_id = %s AND e.scenario_id = %s
        ORDER BY e.start_date
    """
    rows = execute_query(query, (user_id, scenario_id))
    return rows


In [4]:
user_id = 103
scenario_id=29


In [5]:
entries= get_entries(user_id,scenario_id)

In [6]:
import pandas as pd

In [15]:
start_date = date.today()
start_date
time_frame='monthly'
forecast_length=6

In [43]:
def expand_entries(entries, start_date, forecast_length, time_frame="monthly"):
    # Determine end date of forecast
    if time_frame == "daily":
        end_date = start_date + timedelta(days=forecast_length-1)
    elif time_frame == "weekly":
        end_date = start_date + timedelta(weeks=forecast_length-1)
    elif time_frame == "monthly":
        end_date = start_date + relativedelta(months=forecast_length-1)
    elif time_frame == "quarterly":
        end_date = start_date + relativedelta(months=3*forecast_length-1)
    elif time_frame == "yearly":
        end_date = start_date + relativedelta(years=forecast_length-1)
    else:
        raise ValueError("Invalid time_frame")
    all_occurrences = []
    for e in entries:
        freq= e['entry_frequency']
        e_start=e['entry_start_date']
        e_end = e['entry_end_date']
        e_name = e['entry_name']
        print(f"Expanding entry {e_name}: {e_start}-{e_end} ({freq})")
        if freq =='one_time':
            if start_date <= e_start <= end_date:
                print(f"Adding entry {e_name}")
                all_occurrences.append({**e,"occurrence_date":e_start})
            continue
        freq_map = {
                "daily": relativedelta(days=1),
                "weekly": relativedelta(weeks=1),
                "monthly": relativedelta(months=1),
                "quarterly": relativedelta(months=3),
                "yearly": relativedelta(years=1),
            }
        delta = freq_map[freq]
        current = e_start
        # If entry end date is within the period end date and current date is before the entry_end
        while (e_end is None or current <= e_end)  and current <= end_date:
            print(f"")
            if current >= start_date:
                print(f"Adding entry {e_name} for date {current}")
                all_occurrences.append({**e,"occurrence_date":current})
            current += delta
    df = pd.DataFrame(all_occurrences)
    if df.empty:
        return df

    # Annotate with calendar info
    df["occurrence_date"] = pd.to_datetime(df["occurrence_date"], errors="coerce")
    df["year"] = df["occurrence_date"].apply(lambda d: d.year)
    df["month"] = df["occurrence_date"].apply(lambda d: d.month)
    df["day"] = df["occurrence_date"].apply(lambda d: d.day)
    df["week_no"] = df["occurrence_date"].apply(lambda d: d.isocalendar()[1])
    
    return df

def aggregate_forecast(df, time_frame="monthly"):
    if df.empty:
        return []

    if time_frame == "daily":
        grouped = df.groupby("occurrence_date")
    elif time_frame == "weekly":
        grouped = df.groupby(["year", "week_no"])
    elif time_frame == "monthly":
        grouped = df.groupby(["year", "month"])
    elif time_frame == "quarterly":
        grouped = df.groupby([pd.Grouper(key="occurrence_date", freq="Q")])
    elif time_frame == "yearly":
        grouped = df.groupby("year")
    else:
        raise ValueError("Invalid time_frame")

    forecast = []
    for period, group in grouped:
        profit_loss = group.apply(
            lambda row: row["entry_amount"] if row["entry_type"] == "income" else -row["entry_amount"], axis=1
        ).sum()
        period_start = group["occurrence_date"].min()
        period_end = group["occurrence_date"].max()
        forecast.append({
            "period_start": period_start,
            "period_end": period_end,
            "entries": group.to_dict(orient="records"),
            "profit_loss": profit_loss
        })

    return forecast

    



In [ ]:
expanded_entries = expand_entries(entries,start_date,12,time_frame)


In [44]:
grouped= aggregate_forecast(expanded_entries,time_frame)

In [45]:
grouped

[{'period_start': Timestamp('2026-01-10 00:00:00'),
  'period_end': Timestamp('2026-01-27 00:00:00'),
  'entries': [{'entry_id': 33,
    'entry_name': 'Car Insurance',
    'entry_type': 'expense',
    'entry_frequency': 'monthly',
    'entry_start_date': datetime.date(2026, 1, 10),
    'entry_end_date': datetime.date(2026, 2, 10),
    'entry_amount': Decimal('140.00'),
    'entry_scenario_id': 29,
    'entry_category_id': 52,
    'category_id': 52,
    'category_name': 'Car Insurance',
    'occurrence_date': Timestamp('2026-01-10 00:00:00'),
    'year': 2026,
    'month': 1,
    'day': 10,
    'week_no': 2},
   {'entry_id': 28,
    'entry_name': 'Rent',
    'entry_type': 'expense',
    'entry_frequency': 'monthly',
    'entry_start_date': datetime.date(2026, 1, 27),
    'entry_end_date': datetime.date(2026, 1, 27),
    'entry_amount': Decimal('1000.00'),
    'entry_scenario_id': 29,
    'entry_category_id': 51,
    'category_id': 51,
    'category_name': 'Rent',
    'occurrence_date': 